In [2]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from lightgbm import LGBMRegressor

In [3]:
ct_df = pd.read_csv('../data/results/ct_predicted_value')
t_df = pd.read_csv('../data/results/t_predicted_value')

In [5]:
ct_df.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
files = ct_df['file'].unique()

nxt_rnd_ct_val = []

log = 0
for file in files:
    
    ct_df_file = ct_df[ct_df['file'] == file]
    rounds = ct_df_file['round'].unique()
    log += 1
    if log % 500 == 0:
        print(f'{log}/12185')
    
    for rnd in rounds:
        if rnd == rounds[-1]:
            nxt_rnd_ct_val.append(0)
        else:
            nxt_rnd_ct_val.append(ct_df_file[ct_df_file['round'] == rnd +1]['ct_val_real'].values[0])

500/12185
1000/12185
1500/12185
2000/12185
2500/12185
3000/12185
3500/12185
4000/12185
4500/12185
5000/12185
5500/12185
6000/12185
6500/12185
7000/12185
7500/12185
8000/12185
8500/12185
9000/12185
9500/12185
10000/12185
10500/12185
11000/12185
11500/12185
12000/12185


In [9]:
ct_df['nxt_rnd_ct_val'] = nxt_rnd_ct_val

In [11]:
ct_df.drop(columns=['ct_val_real'], inplace=True)

In [12]:
ct_df.head()

,file,round,wp_ct_val,nade_ct_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,ct_val_pred,nxt_rnd_ct_val
0,0,1,1000.0,550,5,5,0.5,0.5,0,0,4078.134589,18450
1,0,2,10100.0,1100,4,0,1.0,0.0,1,0,17819.702711,9550
2,0,3,4125.0,900,0,1,0.0,0.0,0,1,7038.468589,1600
3,0,4,1000.0,0,0,3,0.0,1.0,0,2,1452.468928,23350
4,0,5,15500.0,1400,0,4,0.0,1.0,0,3,22676.205763,26400


In [14]:
FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 
         'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 
         'ct_val_pred']

TARGET = ['nxt_rnd_ct_val']

In [15]:
numeric_transformer = Pipeline(steps=[('sacler', StandardScaler())])

In [16]:
preprocessor = ColumnTransformer(transformers=[('num', numeric_transformer, FEATS)])

In [17]:
ct_train, ct_test = train_test_split(ct_df)

In [18]:
print(ct_train.shape, ct_test.shape)

(233109, 12) (77704, 12)


In [19]:
regressor = LGBMRegressor()

In [20]:
model = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', regressor)])

In [21]:
model.fit(ct_train[FEATS], ct_train[TARGET])

/home/victor/miniconda3/envs/ironhack_env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('sacler',
                                                                   StandardScaler())]),
                                                  ['file', 'round', 'wp_ct_val',
                                                   'nade_ct_val', 'ct_alive',
                                                   't_alive', 'ct_winner',
                                                   'bomb_planted',
                                                   'ct_cons_wins',
                                                   't_cons_wins',
                                                   'ct_val_pred'])])),
                ('regressor', LGBMRegressor())])

In [22]:
y_ct_train = model.predict(ct_train[FEATS])

In [24]:
r2_score(y_true = ct_train[TARGET], y_pred = y_ct_train)

0.3527883569907234